In [ ]:
import os
import sys
import warnings

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy import table
from astropy.io import fits
import astropy.units as u
from astropy.time import Time
from astropy.utils.exceptions import ErfaWarning
from matplotlib import transforms

In [ ]:
from sherpa.astro import ui
import sherpa

In [ ]:
import ChiantiPy.core as ch

In [ ]:
ui.set_conf_opt("sigma", 1.645)
ui.set_conf_opt("numcores", 3)
ui.set_xsabund("aspl")

In [ ]:
delta_E = 0.05
line_energies = [(1.85, 'Si XIII'), (2.01, 'Si XIV'),
                 (3.88, 'Ca XIX'), (4.11, 'Ca XX'),
                 (6.7, 'Fe XXV'),
                ]
axranges = [(.5, 10.), (1.6, 2.2), (3.7, 4.3), (6, 7.2)]

def specplot(plotfunc, figsize=None):
    axes = []
    fig = plt.figure(figsize=figsize)
    axes.append(fig.add_subplot(2, 1, 1))
    axes.append(fig.add_subplot(2, 3, 4))
    axes.append(fig.add_subplot(2, 3, 5, sharey=axes[1]))
    axes.append(fig.add_subplot(2, 3, 6, sharey=axes[1]))
    
    for i, ax in enumerate(axes):
        plt.sca(ax)
        plotfunc()
        trans = transforms.blended_transform_factory(ax.transData, ax.transAxes)

        for x in line_energies:
            # Mark and label only lines in range
            if (x[0] > axranges[i][0]) & (x[0] < axranges[i][1]):
                ax.axvspan(x[0] - delta_E, x[0] + delta_E, facecolor='0.5', alpha=0.5)
                # in top figure, not enough space
                if i == 0:
                    ax.text(x[0], 0.9, x[1].split(' ')[0], horizontalalignment='center',
                        verticalalignment='center', transform=trans)
                else:
                    ax.text(x[0], 0.9, x[1], horizontalalignment='center',
                        verticalalignment='center', transform=trans)
        ax.set_title('')
        ax.set_xlim(*axranges[i])
        
    axes[0].semilogy()
    axes[0].set_ylim(1e-5, None)
    for i in [2, 3]:
        plt.setp(axes[i].get_yticklabels(), visible=False)
        axes[i].set_ylabel('')
        
    for ax in axes[1:]:
        zoom_effect(ax, axes[0], color='0.5')
    return fig, axes

In [ ]:
data = Table()
data['ObsID'] = ['14539', '17644', '17764', '19980', '21176', '22323', '23100', '23102', '23101',
                 '17764 + 19980', '22323+23100+23101+23102']
data['filestem'] = list(data['ObsID'][:9]) + ['2017', '2019']
data['filesrcB'] = list([ n + '_B_grp.pi' for n in data['ObsID'][:9]]) + ['2017_B_src.pi', '2019_B_src.pi']
data['filebkgB'] = [ n + '_B_bkg.pi' for n in data['filestem']]
data['filesrcA'] = [n.replace('B', 'A') for n in data['filesrcB']]
data['filebkgA'] = [n.replace('B', 'A') for n in data['filebkgB']]
data['year'] = [fits.getval('data/Chandra/' + n, 'DATE-OBS')[:4] for n in data['filesrcA'][:9]] + ['2017', '2019']

plotorder = [0, 1, 9, 4, 10]

In [ ]:
#Move up, use data[year] for all plots
mplcolors = plt.rcParams['axes.prop_cycle'].by_key()['color']
mplcolors = dict(zip(sorted(set(data['year'])), mplcolors))

data['color'] = [mplcolors[r['year']] for r in data]

In [ ]:
data

In [ ]:
def read_lcs(obsid, source):
    lcall = Table.read('data/Chandra/{0}_{1}_lc.fits'.format(obsid, source), hdu=1)
    lcsoft = Table.read('data/Chandra/{0}_{1}_lc_soft.fits'.format(obsid, source), hdu=1)
    lchard = Table.read('data/Chandra/{0}_{1}_lc_hard.fits'.format(obsid, source), hdu=1)
    lc = table.hstack([lcall, lcsoft, lchard], table_names=['all', 'soft', 'hard'], metadata_conflicts='silent')
    # time columns are the same for each lightcurve, so remove dublicate entries here for simplicity
    for c in lc.colnames:
        if (('TIME' in c) or ('AREA' in c) or ('EXPOSURE' in c)) and ('all' in c):
            lc.rename_column(c, c[:-4])
            lc.remove_columns([c[:-4] + '_soft', c[:-4] + '_hard'])
    ind  = lc['EXPOSURE'] > 0.
    return lc[ind]
    
lccurves = [[ read_lcs(obsid, t) for t in ['srca', 'srcb']] for obsid in data['ObsID'][:-2]]
for list1 in lccurves:
    for lc in list1:
        lc['t'] = lc['TIME'] - lc['TIME'][0]

In [ ]:
lcaca = [Table.read('data/Chandra/monitor_{0}_lc.fit'.format(obsid), hdu=1)
           for obsid in data['ObsID'][1:-2]]

In [ ]:
fig = plt.figure(figsize=(8, 13))

nlc = len(lccurves)

width = np.array([lc[0]['t'][-1] for lc in lccurves])
width = width / width.sum() * 0.8  # last factor is scale factor to make space for label left of plot
dy = [0.11, 0.11, 0.06]
ypos = [0.1, 0.21, 0.32]
axes = []
for y in range(3):
    for x in range(len(lccurves)):
        kwargs = {}
        if y != 0:
            kwargs['sharex'] = axes[x]
        if x != 0:
            kwargs['sharey'] = axes[y * nlc]
        axes.append(fig.add_axes((.1 + np.sum(width[0:x]) + x * 0.02, ypos[y], width[x], dy[y]), **kwargs))

for j in range(3):
    for i in range(1, nlc):
        plt.setp(axes[j * nlc + i].get_yticklabels(), visible=False)
        
for j in [1, 2]:
    for i in range(nlc):
        plt.setp(axes[j * nlc + i].get_xticklabels(), visible=False)
    
for i, obsid in enumerate(data['ObsID'][:-2]):
    axes[0 * nlc + i].errorbar(lccurves[i][0]['t']/1e3, lccurves[i][0]['NET_RATE_all'] * 1e3, lccurves[i][0]['ERR_RATE_all'] * 1e3, 
                             label='0.3-9.0 keV', color='b', lw=5, alpha=0.3)
    axes[0 * nlc + i].plot(lccurves[i][0]['t']/1e3, lccurves[i][0]['NET_RATE_soft'] * 1e3, 
                         color='b', ls=':', label='0.3-1.0 keV')
    axes[0 * nlc + i].plot(lccurves[i][0]['t']/1e3, lccurves[i][0]['NET_RATE_hard'] * 1e3, 
                         color='b', ls='--', label='1.0-9.0 keV')
    axes[1 * nlc + i].errorbar(lccurves[i][1]['t']/1e3, lccurves[i][1]['NET_RATE_all'] * 1e3, lccurves[i][1]['ERR_RATE_all'] * 1e3,
                             label='0.3-9.0 keV', color='g', lw=5, alpha=0.3)
    axes[1 * nlc + i].plot(lccurves[i][1]['t']/1e3, lccurves[i][1]['NET_RATE_soft'] * 1e3, 
                         color='g', ls=':', label='0.3-1.0 keV')
    axes[1 * nlc + i].plot(lccurves[i][1]['t']/1e3, lccurves[i][1]['NET_RATE_hard'] * 1e3, 
                     color='g', ls='--', label='1.0-9.0 keV')
    axes[2 * nlc + i].set_title(obsid)
    # ACA data
    if i > 0:
        axes[2 * nlc + i].plot((lcaca[i - 1]['time'] - lcaca[i - 1]['time'][0]) / 1e3, lcaca[i - 1]['mag'], color='k')
   
axes[2 * nlc].invert_yaxis()


In [ ]:
for obsid in data['ObsID'][:-2]:
    lcall = Table.read('data/Chandra/{0}_{1}_lc.fits'.format(obsid, 'srca'), hdu=1)
    print(obsid, np.sum(lcall['COUNTS']))

In [ ]:
for row in data:
    ui.load_data(row['filestem'] + '_B', 'data/Chandra/' + row['filesrcB'])
    ui.load_bkg(row['filestem'] + '_B', 'data/Chandra/' + row['filebkgB'])                  
    ui.load_data(row['filestem'] + '_A', 'data/Chandra/' + row['filesrcA'])
    ui.load_bkg(row['filestem'] + '_A', 'data/Chandra/' + row['filebkgA'])                     

In [ ]:
ui.notice(None, None)

In [ ]:
ui.get_filter('22323_A')

In [ ]:
for o in ui.list_data_ids():
    ui.group_width(o, 1)

In [ ]:
ui.set_analysis('energy')
ui.ignore(None, 0.3)
ui.ignore(9., None)

In [ ]:
ui.get_filter('22323_A')

### AAVSO data

In [ ]:
aavso = Table.read('data/aavso.txt', format='ascii.csv', fill_values = ('N/A', 0))
# Sometimes the Magnitude column contains the "<" sing for upper limits.
aavso['Mag'] = np.zeros(len(aavso))
for i in range(len(aavso)):
    try:
        aavso['Mag'][i] = float(aavso['Magnitude'][i])
    except ValueError:
        aavso['Mag'][i] = np.ma.masked

In [ ]:
bands = dict([('Vis.', {'color': (1., 0.5, 1.), 'marker': '.'}),
                     ('B', {'color': 'b', 'marker': 'p'}),
                     ('V', {'color': 'g', 'marker': 'o'}),
                     ('R', {'color': 'r', 'marker': '*'}),
                     ('ACA', {'color': 'k', 'marker': 'D', 'markersize': 7})
                    ])
def plotaavso(ax):
    for band in bands:
        ind = (aavso['Band'] == band)
        ax.plot(aavso['JD'][ind]-2400000.5, aavso['Mag'][ind], linestyle='None', **bands[band], label=band)

In [ ]:
obstimes = np.array([[Time(lc[0]['TIME_MIN'][0], format='cxcsec').mjd, 
             Time(lc[0]['TIME_MAX'][-1], format='cxcsec').mjd] for lc in lccurves])

In [ ]:
# Add Chandra ACA magnitudes
for l in lcaca:
    aavso.add_row({'JD': Time(l['time'].mean(), format='cxcsec').jd, 'Mag': l['mag'].mean(), 'Band': 'ACA'})

In [ ]:
def years(tmjd):
    return Time(tmjd, format='mjd').decimalyear

def update_axyears(ax):
   y1, y2 = ax.get_xlim()
   axy.set_xlim(years(y1), years(y2))
   ax.figure.canvas.draw()

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
axy = ax.twiny()
# automatically update ylim of ax2 when ylim of ax1 changes.
ax.callbacks.connect("xlim_changed", update_axyears)

plotaavso(ax)
ax.set_xlim([56000, 59000])
ax.invert_yaxis()

ax.eventplot(obstimes[:, 0], colors=['k'], lineoffsets=[9],
                    linelengths=[1])

In [ ]:
from matplotlib.ticker import MaxNLocator, MultipleLocator
    
from matplotlib.transforms import Bbox, TransformedBbox, \
    blended_transform_factory, Affine2D

from mpl_toolkits.axes_grid1.inset_locator import BboxPatch, BboxConnector,\
    BboxConnectorPatch

# code for zoom effects taken and modified from http://matplotlib.org/users/annotations_guide.html
def connect_bbox(bbox1, bbox2,
                 loc1a, loc2a, loc1b, loc2b,
                 prop_lines, prop_patches=None):
    if prop_patches is None:
        prop_patches = prop_lines.copy()
        prop_patches["alpha"] = prop_patches.get("alpha", 1)*0.2

    c1 = BboxConnector(bbox1, bbox2, loc1=loc1a, loc2=loc2a, **prop_lines)
    c1.set_clip_on(False)
    c2 = BboxConnector(bbox1, bbox2, loc1=loc1b, loc2=loc2b, **prop_lines)
    c2.set_clip_on(False)

    bbox_patch1 = BboxPatch(bbox1, **prop_patches)
    bbox_patch2 = BboxPatch(bbox2, **prop_patches)

    p = BboxConnectorPatch(bbox1, bbox2,
                           # loc1a=3, loc2a=2, loc1b=4, loc2b=1,
                           loc1a=loc1a, loc2a=loc2a, loc1b=loc1b, loc2b=loc2b,
                           **prop_patches)
    p.set_clip_on(False)

    return c1, c2, bbox_patch1, bbox_patch2, p


def zoom_effect(ax1, ax2, xtrans=Affine2D(), **kwargs):
    """
    ax1 : the main axes
    ax2 : the zoomed axes

    connect ax1 & ax2. The x-range of (xmin, xmax) in both axes will
    be marked.  The keywords parameters will be used ti create
    patches. The xmin & xmax will be taken from the
    ax1.viewLim.
    """

    tt = ax1.transScale + (ax1.transLimits + ax2.transAxes)
    trans = blended_transform_factory(xtrans + ax2.transData, tt)

    mybbox1 = ax1.bbox
    mybbox2 = TransformedBbox(ax1.viewLim, trans)

    prop_patches = kwargs.copy()
    prop_patches["ec"] = "none"
    prop_patches["alpha"] = 0.2

    # The inversion messes up the coner points.
    # This is tweaked for inverted axes.
    # I'm sure ther is some clever way to use the right transforms for the y axis 
    # to take care of that automatically, but for 
    # a one-off script it's much simpler to just adjust the corners by hand.
    if ax1.yaxis_inverted():
        loc2a = 2
        loc2b = 1
    else:
        loc2a = 3
        loc2b = 4
    
    c1, c2, bbox_patch1, bbox_patch2, p = \
        connect_bbox(mybbox1, mybbox2,
                     loc1a=2, loc2a=loc2a, loc1b=1, loc2b=loc2b,
                     prop_lines=kwargs, prop_patches=prop_patches)

    #ax1.add_patch(bbox_patch1)
    ax2.add_patch(bbox_patch2)
    ax2.add_patch(c1)
    ax2.add_patch(c2)
    ax2.add_patch(p)

    return c1, c2, bbox_patch1, bbox_patch2, p

In [ ]:
obsids = data['ObsID'][:-2]
n_obsids = len(obsids)
year = np.array(list(set([int(y) for y in data['year']])))
year.sort()
n_years = len(year)

fig = plt.figure(figsize=(8, 13))
ax00 = fig.add_subplot(511)
ax11 = fig.add_subplot(512)

n_years = len(set(data['year']))
axrow2 = [fig.add_subplot(5, n_years, 2 * n_years + 1)]
for i in range(2, n_years+1):
    axrow2.append(fig.add_subplot(5, n_years, 2 * n_years + i, sharey=axrow2[0]))

# Set up second x-axis for top plot that is labeled in years
ax00years = ax00.twiny()
ax11years = ax11.twiny()

def years(tmjd):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', ErfaWarning)
        t = Time(tmjd, format='mjd').decimalyear
    return t

def update_ax11years(ax11):
   y1, y2 = ax11.get_xlim()
   ax11years.set_xlim(years(y1), years(y2))
   ax11.figure.canvas.draw()

def update_ax00years(ax00):
   y1, y2 = ax00.get_xlim()
   ax00years.set_xlim(years(y1), years(y2))
   ax00.figure.canvas.draw()

# automatically update ylim of ax2 when ylim of ax1 changes.
ax00.callbacks.connect("xlim_changed", update_ax00years)
ax11.callbacks.connect("xlim_changed", update_ax11years)

for ax in fig.axes:
    plotaavso(ax)
    
for ax in fig.axes:
    for i in range(obstimes.shape[0]):
        ax.bar(obstimes[i, 0], height=10, width=obstimes[i, 1]-obstimes[i, 0], bottom=8, align='edge', 
               color='0.5', edgecolor='0.5')

ax00.invert_yaxis()
ax11.invert_yaxis()
ax00.set_xlim([52000, 59000])
ax11.set_xlim([56200, 59000])

dtime = 15.
for i, y in enumerate(year):
    meanmjd = np.mean(obstimes[data['year'][:-2] == str(y)])
    axrow2[i].set_xlim(meanmjd - dtime, meanmjd + dtime)
    axrow2[i].get_xaxis().get_major_formatter().set_useOffset(False)
    axrow2[i].xaxis.set_major_locator( MaxNLocator(nbins=3, steps=[5, 10]) )
    if i > 0:
        plt.setp(axrow2[i].get_yticklabels(), visible=False)
    #axrow2[i].set_title(y)
        
axrow2[0].invert_yaxis()

ax00.legend(numpoints=1, loc='lower left')
ax11.set_ylabel('mag')
ax21.set_ylabel('mag')
ax22.set_xlabel('time [MJD]')
ax00years.set_xlabel('time [years]')

axrow2[0].set_ylim([13.5, 9])
ax11.set_ylim([15, 9])
ax00.set_ylim([15, 9])

width = np.array([lc[0]['t'][-1] for lc in lccurves])
width = width / width.sum() * 0.7  # last factor is scale factor to make space for label left of plot
dy = [0.11, 0.11, 0.06]
ypos = [0.1, 0.21, 0.32]
axes = []


for y in range(3):
    for x in range(n_obsids):
        kwargs = {}
        if y != 0:
            kwargs['sharex'] = axes[x]
        if x != 0:
            kwargs['sharey'] = axes[y * n_obsids]
        axes.append(fig.add_axes((.1 + np.sum(width[0:x]) + x * 0.02, ypos[y], width[x], dy[y]), **kwargs))
        
for j in range(3):
    for i in range(1, n_obsids):
        plt.setp(axes[j * n_obsids + i].get_yticklabels(), visible=False)
        
for j in [1, 2]:
    for i in range(n_obsids):
        plt.setp(axes[j * n_obsids + i].get_xticklabels(), visible=False)

for i, obsid in enumerate(obsids):
    axes[0 * n_obsids + i].errorbar(lccurves[i][0]['t']/1e3, lccurves[i][0]['NET_RATE_all'] * 1e3, lccurves[i][0]['ERR_RATE_all'] * 1e3, 
                             label='0.3-9.0 keV', color='b', lw=5, alpha=0.3)
    axes[0 * n_obsids + i].plot(lccurves[i][0]['t']/1e3, lccurves[i][0]['NET_RATE_soft'] * 1e3, 
                         color='b', ls=':', label='0.3-1.0 keV')
    axes[0 * n_obsids + i].plot(lccurves[i][0]['t']/1e3, lccurves[i][0]['NET_RATE_hard'] * 1e3, 
                         color='b', ls='--', label='1.0-9.0 keV')
    axes[1 * n_obsids + i].errorbar(lccurves[i][1]['t']/1e3, lccurves[i][1]['NET_RATE_all'] * 1e3, lccurves[i][1]['ERR_RATE_all'] * 1e3,
                             label='0.3-9.0 keV', color='g', lw=5, alpha=0.3)
    axes[1 * n_obsids + i].plot(lccurves[i][1]['t']/1e3, lccurves[i][1]['NET_RATE_soft'] * 1e3, 
                         color='g', ls=':', label='0.3-1.0 keV')
    axes[1 * n_obsids + i].plot(lccurves[i][1]['t']/1e3, lccurves[i][1]['NET_RATE_hard'] * 1e3, 
                     color='g', ls='--', label='1.0-9.0 keV')
    #axes[2 * 4 + i].set_title(obsid)
    if i > 0:
        axes[2 * n_obsids + i].plot((lcaca[i-1]['time'] - lccurves[i][0]['TIME'][0]) / 1e3, # Subtract 0 point of X-ray lc
                                    lcaca[i - 1]['mag'], color='k')
    
#axes[n_obsids * 2].yaxis.set_major_locator( MaxNLocator(nbins=4) )
axes[8].legend()
axes[n_obsids * 2].invert_yaxis()
# axes[n_obsids + 8].legend()
axes[0].set_ylabel('              net X-ray counte rate [cts/ks]')
axes[4].set_xlabel('time from beginning of observation [ks]')
axes[n_obsids * 2].set_ylabel('ACA [mag]')
axes[n_obsids].text(0., 60., 'RW Aur B')
axes[0].text(0, 2., 'RW Aur A')
axes[n_obsids * 2].text(0, 11.1, 'RW Aur A + B\n(unresolved)')
axes[n_obsids * 2].text(0, 12, 'no ACA data', color='0.5')
#axes[n_obsids * 2].text(35., 11, '2013-Jan')
#axes[n_obsids * 2 + 1].text(15, 11., '2015-Apr')
#axes[n_obsids * 2 + 2].text(15, 11.5, '2017-Jan-09')
#axes[n_obsids * 2 + 3].text(-2, 11.5, '2017\nJan-11')

zoom_effect(ax11, ax00, color='0.5')
for ax in axrow2:
    zoom_effect(ax, ax11, color='0.5')
for i in range(n_obsids):
    zoom_effect(axes[2 * n_obsids + i], axrow2[np.nonzero(year == int(data['year'][i]))[0][0]], 
                xtrans=Affine2D.from_values(1./(24*3600/1000), 0, 0, 1, obstimes[i, 0], 0), color='0.5')
               
                   
fig.subplots_adjust(wspace=0.02, hspace=.35, left=0.1, right=0.95)

#fig.savefig(os.path.join(figout, 'lc.pdf'), bbox_inches='tight')
#fig.savefig(os.path.join(figout, 'lc.png'), bbox_inches='tight', dpi=600)

## The spectrum of RW Aur B
This needs tobe modeled, because we will use it as a component in the RW Aur A model to account for the contamination of the RW Aur A data by signal coming from RW Aur B.

In [ ]:
for i in ui.list_data_ids():
    if i[-2:] == '_B':
        ui.group_counts(i, 15)
    
for i in plotorder:
    ui.plot_data(data['filestem'][i] + '_B', overplot=True, color=data['color'][i])
    
ax = plt.gca()
ax.loglog()
ax.set_xlim(.4, 6)
ax.set_ylim(2e-4, 0.1)
ax.legend(ax.get_lines(), data['year'][plotorder])
ax.set_title('RW Aur B')

In [ ]:
years = list(set(data['year']))
years.sort()

b_models = {y: ui.xsphabs(name='Ba_'+y) * (ui.xsvapec(name='Bv1_'+y) + ui.xsvapec(name='Bv2_'+y)) for y in years}

for row in data:
    ui.set_source(row['filestem'] + '_B', b_models[row['year']])

In [ ]:
for bo in b_models.values():
    for elem in ['C', 'N', 'O','Ne', 'Fe', 'Si', 'Mg']:
        setattr(bo.rhs.rhs, elem, getattr(bo.rhs.lhs, elem))
    bo.rhs.lhs.kT = 0.5
    bo.rhs.rhs.kT = 2.0

In [ ]:
ui.show_model()

In [ ]:
to_fit_B = [row['filestem']+'_B' for row in data if row['filestem']==row['ObsID']]

In [ ]:
# Leaving these independent can actually lead to the worse chi^2, so no reason to suspect that nH is changeing
# Fix to reduce number of parameters in fit

for y in years[1:]:
    b_models[y].lhs.nH = b_models[years[0]].lhs.nH

ui.fit(*to_fit_B)

In [ ]:
for row in data:
    ui.subtract(row['filestem']+'_B')

In [ ]:
# It seems that joining the temperature of both components should work well
for y in years[1:]:
    b_models[y].rhs.lhs.kT = b_models[years[0]].rhs.lhs.kT
    b_models[y].rhs.rhs.kT = b_models[years[0]].rhs.rhs.kT
    
ui.fit(*to_fit_B)

The spectra are so remarkebly similar, it's almost uncanning, given the change in flux that I see in the lightcurve. It seems that all the variability I see within one observation washes out when I add the data from the entire observation together. It might actually we worth investigating, if there are any changes at all! The norms are significantly different, though.

I tried freeing abundances, but this is so close to defaults, there really is no reason to set them to anything else than 1.

In [ ]:
# Use the following line to change the size of the figure
# fig = plt.figure(figsize=(15,10))

for i,j  in enumerate(plotorder):
    ui.plot_fit(data['filestem'][j] + '_B', overplot=not (i==0), color=data['color'][j])
    
ax = plt.gca()
ax.loglog()
ax.set_xlim(.4, 6)
ax.set_ylim(2e-4, 0.1)
ax.legend(ax.get_lines()[::2], data['year'][plotorder])
ax.set_title('RW Aur B')

In [ ]:
# Use the following line to change the size of the figure
fig = plt.figure(figsize=(15,5))

for i,j  in enumerate(plotorder):
    ui.plot_bkg(data['filestem'][j] + '_B', overplot=not (i==0), color=data['color'][j])
    
ax = plt.gca()
ax.loglog()
ax.set_xlim(.4, 9)
ax.set_ylim(1e-5, 0.01)
ax.legend(ax.get_lines(), data['year'][plotorder])
ax.set_title('Bkg for RW Aur B')

## The PSF of RW Aur B

Te PSF might be different from observation to observation for several reasons. In particular, the ACA monitoring may lead to a degradation of the PSF. A larger PSF means that more photons from RW Aur B will contaminate the extraction region of RW Aur A. Thus, I first try to get a quantitative measure of the PSF.

In [ ]:
radprofs = [Table.read('data/Chandra/{0}_rprofile.fits'.format(o), hdu=1) for o in data['filestem'][:-2]]

In [ ]:
for r in radprofs:
    plt.semilogy(r['SUR_BRI'] / r['SUR_BRI'][0], label=r.meta['OBS_ID'])
plt.ylim(.001, 1)
plt.legend()


The observations taken in 2019 (22323, 23100, 23101, 23102) are significantly above the olders ones in annulus 3-6, which is the area overlapping with source A and the area in which we calculate the background for A. It's not much in absolute terms, but it cearly means that the scaling factor between B spectrum as background for A must be different in 2019 and the previous years.

## The spectrum of RW Aur A

In [ ]:
#fig = plt.figure(figsize=(12, 10))
for i in ui.list_data_ids():
    if i[-2:] == '_A':
        ui.group_counts(i, 5)
    
for i in plotorder:
    ui.plot_data(data['filestem'][i] + '_A', overplot=not(i==plotorder[0]), color=data['color'][i])
    
ax = plt.gca()
ax.loglog()
ax.set_xlim(.5, 9)
ax.set_ylim(5e-5, 0.04)
ax.legend(ax.get_lines(), data['year'][plotorder])
ax.set_title('RW Aur A')

In [ ]:
ui.get_filter('22323_A')

In [ ]:
to_fit_A = [row['filestem']+'_A' for row in data if row['filestem']==row['ObsID']]

In [ ]:
scaleB_early = ui.scale1d(name='scaleB_early')
scaleB_2019 = ui.scale1d(name='scaleB_2019')

scaleB = {'2013': scaleB_early, '2015': scaleB_early, '2017': scaleB_early, '2018': scaleB_early, 
          '2019': scaleB_2019}

# Freeze everything in B models except the scale
for row in data:
    b_mod = b_models[row['year']]
    ui.set_bkg_source(row['filestem'] + '_A', scaleB[row['year']] * b_mod)
    
    for model in [b_mod.lhs, b_mod.rhs.lhs, b_mod.rhs.rhs]:
        for par in model.pars:
            par.frozen=True

In [ ]:
# Some of the 2019 datasets have too few counts to make a fit at all. So, use merged dataset here.
# If the count numbers are so small, the error is dominated by counting statistics and not by systematics
# of coadding spectra.

# Can revisit this after I find where all the counts went.
ui.fit_bkg('14539_A', '17644_A', '2017_A', '21176_A', '2019_A')

In [ ]:
# Use the following line to change the size of the figure
fig = plt.figure(figsize=(15,5))

for i,j  in enumerate(plotorder):
    ui.plot_bkg_fit(data['filestem'][j] + '_A', overplot=not (i==0), color=data['color'][j])
    
ax = plt.gca()
ax.loglog()
#ax.set_xlim(.4, 9)
ax.set_ylim(1e-6, 0.01)
ax.legend(ax.get_lines()[::2], data['year'][plotorder])
ax.set_title('Bkg for RW Aur A')

Here, I see that the background is described quite well in the early observations, but, even with the higher scaling factor for 2019, the 2019 background is significantly underpredicted on the soft end. That will cause an apparent soft component in the RW Aur A model, that in trouth is just the background. What's surprising is that the shape of the observed background spectrum has not changed at all. ACIS contamination should reduce the number of soft counts detected (and it does in the RW Aur B spectrum), so why is the number of soft counts not going down here? The rate is far too high to be explained with particle background. Is the soft PSF so much wider than the hard PSF?

While an answer would be good, really what I need is a model that describes the background well enough so I can carry on with fitting. On the other hand, I'm reluctant to introduce a new component that's fit to just a few bins.

In [ ]:
ids = ['22323_A', '23100_A', '23101_A', '23102_A'] #, '2019_A']
for o in ids:
    ui.plot_bkg_fit(o, overplot=True)
ax = plt.gca()
ax.loglog()
#ax.set_xlim(.4, 9)
#ax.set_ylim(1e-5, 0.01)
ax.legend(ax.get_lines()[::2], ids)

In [ ]:
for row in data:
    ui.unsubtract(row['filestem']+'_B')

for o in ui.list_data_ids():
    ui.group_width(o, 1)
    ui.set_analysis(o, "bin", "counts", 0)
    pl = ui.get_data_plot(o)
    pl1 = ui.get_bkg_plot(o)
    print('{}: {} - {}'.format(o, pl.y.sum(), pl1.y.sum()))
    
ui.set_analysis("energy")

In [ ]:
scaleB_early.c0.frozen = True
scaleB_2019.c0.frozen = True

### Line-diagnostics
Of course, we do not really resolve individual lines in ACIS spectra, but below it looks like I should. So, at the very least, I can use CHIANTI to see H/He-like line ratios.

In [ ]:
logtemp = np.arange(5.8, 8.5, .1)
temp = 10**logtemp

def intensityHHe(temp, Hion, Heion):
    Helike = ch.ion(Heion, temperature=temp, eDensity=1.e+9, em=1.e+27)
    Helike.intensity()
    Hlike = ch.ion(Hion, temperature=temp, eDensity=1.e+9, em=1.e+27)
    Hlike.intensity()

    e_r = Helike.Intensity['intensity'][:, (Helike.Intensity['lvl1'] == 1) & (Helike.Intensity['pretty2'] == '1s.2p 1P1.0')].flatten()
    e_i1 = Helike.Intensity['intensity'][:, (Helike.Intensity['lvl1'] == 1) & (Helike.Intensity['pretty2'] == '1s.2p 3P2.0')].flatten()
    e_i2 = Helike.Intensity['intensity'][:, (Helike.Intensity['lvl1'] == 1) & (Helike.Intensity['pretty2'] == '1s.2p 3P1.0')].flatten()
    e_f = Helike.Intensity['intensity'][:, (Helike.Intensity['lvl1'] == 1) & (Helike.Intensity['pretty2'] == '1s.2s 3S1.0')].flatten()

    e_lya1 = Hlike.Intensity['intensity'][:, (Hlike.Intensity['lvl1'] == 1) & (Hlike.Intensity['pretty2'] == '2s 2S0.5')].flatten()
    e_lya2 = Hlike.Intensity['intensity'][:, (Hlike.Intensity['lvl1'] == 1) & (Hlike.Intensity['pretty2'] == '2p 2P0.5')].flatten()
    e_lya3 = Hlike.Intensity['intensity'][:, (Hlike.Intensity['lvl1'] == 1) & (Hlike.Intensity['pretty2'] == '2p 2P1.5')].flatten()
    
    return e_lya1 + e_lya2 + e_lya3, e_r + e_i1 + e_i2 + e_f

In [ ]:
fig, ax = plt.subplots()
axt = ax.twinx()
    
for Heion, Hion, color in ([('ne_9', 'ne_10', 'b'), ('si_13', 'si_14', 'r'), ('ca_19', 'ca_20', 'g')]):
    e_H, e_He = intensityHHe(temp, Hion, Heion)

    ax.semilogx(temp, e_H / np.max(e_He), color=color)
    ax.semilogx(temp, e_He / np.max(e_He), color=color, ls=':')

    axt.semilogy(temp, e_H / e_He, color=color, lw=5, alpha=.5)

#ax.loglog()
axt.set_ylim(0.1, 10)

In [ ]:
a_models = {y: ui.xsphabs(name='Aa_'+y) * (ui.xsvapec(name='Av1_'+y) + ui.xsvapec(name='Av2_'+y)) for y in years}

for row in data:
    a_mod = a_models[row['year']]
    ui.set_source(row['filestem'] + '_A', a_mod)
    for elem in ['C', 'N', 'O','Ne', 'Mg', 'Al', 'Si', 'S', 'Ar', 'Ca', 'Fe', 'Ni']:
        setattr(a_mod.rhs.rhs, elem, getattr(a_mod.rhs.lhs, elem))
    a_mod.lhs.nH.frozen = False

In [ ]:
for i in ui.list_data_ids():
    if i[-2:] == '_A':
        try:
            ui.ungroup(i)
        except sherpa.utils.err.DataErr:
            # It's already ungrouped
            pass
ui.set_stat('cash')

## 2013 - 2017
This is the data that is covered in previous publications already, so the fits here are mostly reproducing the models published there. I could just hardcode the numbers from the previous fits, but for now I actually run the fits again. Results look consistent, but I should probably check again.

In [ ]:
# Set of kT=20 from Skinner & Guedel for 2013 data, fix kT as in Schneider et al for 2015 data
a_models['2013'].rhs.lhs.Ne.frozen = False
a_models['2013'].rhs.lhs.Fe.frozen = False
a_models['2013'].rhs.rhs.kT.frozen = True
a_models['2013'].rhs.rhs.kT = 20.

a_models['2015'].rhs.lhs.kT = a_models['2013'].rhs.lhs.kT
a_models['2015'].rhs.rhs.kT = a_models['2013'].rhs.rhs.kT

a_models['2017'].rhs.lhs.Fe.frozen = False
a_models['2017'].rhs.rhs.kT.frozen = True
a_models['2017'].rhs.rhs.kT = 20.

# Set models close to final to speed up convergence when running the notebook again
# This also redues the risk of running the fit into non-sensical minimums as I observed
# several times when I experimented with different binning, statistics etc.
a_models['2013'].lhs.nH = 0.3
a_models['2013'].rhs.lhs.kT = 0.4
a_models['2013'].rhs.lhs.norm = 1e-4
a_models['2013'].rhs.rhs.norm = 4e-5

a_models['2017'].lhs.nH = 45
a_models['2017'].rhs.lhs.kT = 1
a_models['2017'].rhs.lhs.Fe = 50

In [ ]:
ui.ignore(None, .4)
ui.ignore(8., None)
        
ui.fit('14539_A')
#ui.fit('17644_A')
ui.fit('17764_A', '19980_A')

In [ ]:
# Freeze parameters that we want to take from the 2013 fit at the values
# fitted above to the 2013 data
a_models['2015'].rhs.lhs.kT.frozen = True
a_models['2015'].rhs.rhs.kT.frozen = True
a_models['2015'].rhs.lhs.kT = a_models['2013'].rhs.lhs.kT
a_models['2015'].rhs.rhs.kT = a_models['2013'].rhs.rhs.kT

ui.fit('17644_A')

In [ ]:
fig, axes= plt.subplots(ncols=2, nrows=2, figsize=(10, 10))

for i in ui.list_data_ids():
    if i[-2:] == '_A':
        ui.group_counts(i, 5)

for ax in axes.flatten():
    plt.sca(ax)
    for i,j  in enumerate(plotorder[:3]):
        ui.plot_fit(data['filestem'][j] + '_A', overplot=not(i==0), 
                    color=data['color'][j], clearwindow=False)
        ui.plot_model_component(data['filestem'][j] + '_A', 
                                a_models[data['year'][j]].lhs * a_models[data['year'][j]].rhs.lhs,
                                overplot=True, color=data['color'][j], linestyle=':')
        ui.plot_model_component(data['filestem'][j] + '_A', 
                                a_models[data['year'][j]].lhs * a_models[data['year'][j]].rhs.rhs,
                                overplot=True, color=data['color'][j], linestyle=':')
    
axes[0, 0].loglog()
axes[0, 0].set_ylim(1e-5, 4e-2)
axes[0, 1].semilogx()
axes[1, 0].set_xlim(3.6, 4.2)
axes[1, 1].set_xlim(6., 7.)
axes[1, 1].legend(ax.get_lines()[::4], data['year'][plotorder[:3]])

In [ ]:
for i in ui.list_data_ids():
    if i[-2:] == '_A':
        ui.group_counts(i, 5)

def pl_13to17():
    for i,j  in enumerate(plotorder[:3]):
        ui.plot_fit(data['filestem'][j] + '_A', overplot=not(i==0), 
                    color=data['color'][j], clearwindow=False)
        ui.plot_model_component(data['filestem'][j] + '_A', 
                                a_models[data['year'][j]].lhs * a_models[data['year'][j]].rhs.lhs,
                                overplot=True, color=data['color'][j], linestyle=':')
        ui.plot_model_component(data['filestem'][j] + '_A', 
                                a_models[data['year'][j]].lhs * a_models[data['year'][j]].rhs.rhs,
                                overplot=True, color=data['color'][j], linestyle=':')
    
fig, axes = specplot(pl_13to17, figsize=(10, 8))
axes[0].legend(axes[0].get_lines()[::4], data['year'][plotorder[:3]])

### 2018

In [ ]:
a_models['2018'].rhs.lhs.Si.frozen = False
a_models['2018'].rhs.lhs.Ca.frozen = False
a_models['2018'].rhs.lhs.Fe.frozen = False

abs1 = a_models['2018'].lhs
xsv1 = a_models['2018'].rhs.lhs
xsv2 = a_models['2018'].rhs.rhs

ui.set_source('21176_A', abs1 * (xsv1 + xsv2))

In [ ]:
xsv1.kT = 1.
xsv2.kT = 5
xsv1.kT.frozen = True
xsv2.kT.frozen = False
xsv1.norm = 1e-5
xsv2.norm = 1e-5
xsv2.norm.frozen=False

In [ ]:
#ui.set_stat('chi2gehrels')
ui.ungroup('21176_A')
#ui.unsubtract('21176_A')
ui.set_stat('cash')
ui.notice(1.5, 10.)
ui.ignore(None, 1.5)
ui.ignore(10., None)
print(ui.get_filter('21176_A'))
ui.fit('21176_A')
ui.group_counts('21176_A', 5)
ui.notice(.5, 10.)
ui.ignore(None, .5)
ui.ignore(10., None)


def pl_21176():
    ui.plot_fit('21176_A', clearwindow=False)
    ui.plot_model_component('21176_A', abs1 * xsv1, overplot=True)
    ui.plot_model_component('21176_A', abs1 * xsv2, overplot=True)
    
fig, axes = specplot(pl_21176, figsize=(10, 8))


In [ ]:
for par in [abs1.nH, xsv1.Si, xsv1.Ca, xsv1.Fe, xsv1.norm, xsv2.kT, xsv2.norm]:
    par.frozen = False

In [ ]:
xsv1.kT.frozen = False

In [ ]:
xsjeta = ui.xsvapec(name='xsjeta')
abs2 = ui.xsphabs('abs2')
ui.set_source('21176_A', xsjeta + abs1 * (xsv1 + xsv2))

In [ ]:
abs2.nH = 0.1
abs2.nH.frozen = False

In [ ]:
ui.set_method('moncar')

In [ ]:
#ui.set_stat('chi2gehrels')
ui.ungroup('21176_A')
#ui.unsubtract('21176_A')
ui.set_stat('cash')
ui.notice(.5, 10.)
ui.ignore(None, .5)
ui.ignore(10., None)
print(ui.get_filter('21176_A'))
ui.fit('21176_A')
ui.group_counts('21176_A', 5)
ui.notice(.5, 10.)
ui.ignore(None, .5)
ui.ignore(10., None)

def pl_21176():
    ui.plot_fit('21176_A', clearwindow=False)
    ui.plot_model_component('21176_A', abs1 * xsv1, overplot=True)
    ui.plot_model_component('21176_A', abs1 * xsv2, overplot=True)
    ui.plot_model_component('21176_A', xsjeta, overplot=True)
    
fig, axes = specplot(pl_21176, figsize=(10, 8))

In [ ]:
abspart = ui.xspcfabs(name='abspart')
abspart.CvrFract.max = 1.
ui.set_source('21176_A', abs1 * xsv1 + abspart * xsv2)

abs1.nH = 19.4
xsv1.kT = 0.6
xsv1.Si = 8.9
xsv1.Ca = 7.7
xsv1.Fe = 1.5
xsv1.norm = 0.015
xsv2.kT = 4.3
xsv2.norm = 0.00085
abspart.nH = 61
abspart.CvrFract = 0.97

In [ ]:
abspart = ui.xspcfabs(name='abspart')
abspart.CvrFract.max = 1.
ui.set_source('21176_A', abs1 * xsv1 + abspart * xsv2)

ui.ungroup('21176_A')
#ui.unsubtract('21176_A')
ui.set_stat('cash')
ui.notice(.5, 10.)
ui.ignore(None, .5)
ui.ignore(10., None)
print(ui.get_filter('21176_A'))
ui.fit('21176_A')
ui.group_counts('21176_A', 3)
ui.notice(.5, 10.)
ui.ignore(None, .5)
ui.ignore(10., None)
   

In [ ]:
                      
def pl_21176():
    ui.plot_fit('21176_A', clearwindow=False)
    ui.plot_model_component('21176_A', abs1 * xsv1, overplot=True)
    ui.plot_model_component('21176_A', abspart * xsv2, overplot=True)
    #ui.plot_model_component('21176_A', xsv1, overplot=True, color='k')
    #ui.plot_model_component('21176_A', xsv2, overplot=True, color='k')
    
fig, axes = specplot(pl_21176, figsize=(10, 8))

In [ ]:
for i in range(len(a_models['2019'].pars)):
    a_models['2019'].pars[i].val = a_models['2018'].pars[i].val
    a_models['2019'].pars[i].frozen = True

In [ ]:
a_models['2019'].rhs.lhs.Ca = 1
a_models['2019'].rhs.lhs.Si = 1
a_models['2019'].rhs.rhs.Ca = 1
a_models['2019'].rhs.rhs.Si = 1

In [ ]:
def pl_2019():
    ui.plot_fit('2019_A', clearwindow=False)
    ui.plot_model_component('2019_A', abs1 * xsv1, overplot=True)
    ui.plot_model_component('2019_A', abs1 * xsv2, overplot=True)
    
fig, axes = specplot(pl_2019, figsize=(10, 8))

In [ ]:
a_models['2019'].rhs.lhs.Si.frozen = True
a_models['2019'].rhs.lhs.Ca.frozen = True
a_models['2019'].rhs.lhs.Fe.frozen = False

abs1 = a_models['2019'].lhs
xsv1 = a_models['2019'].rhs.lhs
xsv2 = a_models['2019'].rhs.rhs

xsv1.kT.frozen = False
xsv2.kT.frozen = False
xsv1.norm.frozen = False
xsv2.norm.frozen = False

xsv1.Si = 1
xsv1.Ca = 1
xsv1.Fe = 5


abspart = ui.xspcfabs(name='abspart')
abspart.CvrFract.max = 1.

for o in ['22323_A', '23100_A', '23101_A', '23102_A', '2019_A']:
    ui.set_source(o, abs1 * xsv1 + abspart * xsv2)

In [ ]:
ui.ungroup('2019_A')
#ui.unsubtract('21176_A')
ui.set_stat('cash')
ui.notice(.5, 10.)
ui.ignore(None, .5)
ui.ignore(10., None)
print(ui.get_filter('2019_A'))
ui.fit('2019_A')
ui.group_counts('2019_A', 3)
ui.notice(.5, 10.)
ui.ignore(None, .5)
ui.ignore(10., None)


In [ ]:
def pl_2019part():
    ui.plot_fit('2019_A', clearwindow=False)
    ui.plot_model_component('2019_A', abs1 * xsv1, overplot=True)
    ui.plot_model_component('2019_A', abspart * xsv2, overplot=True)
    
fig, axes = specplot(pl_2019part, figsize=(10, 8))

In [ ]:
for i in ui.list_data_ids():
    if i[-2:] == '_A':
        ui.group_counts(i, 5)

def pl_13to19():
    for i,j  in enumerate(plotorder):
        ui.plot_fit(data['filestem'][j] + '_A', overplot=not(i==0), 
                    color=data['color'][j], clearwindow=False)
        #ui.plot_model_component(data['filestem'][j] + '_A', 
        #                        a_models[data['year'][j]].lhs * a_models[data['year'][j]].rhs.lhs,
        #                        overplot=True, color=data['color'][j], linestyle=':')
        #ui.plot_model_component(data['filestem'][j] + '_A', 
        #                        a_models[data['year'][j]].lhs * a_models[data['year'][j]].rhs.rhs,
        #                        overplot=True, color=data['color'][j], linestyle=':')
    
fig, axes = specplot(pl_13to19, figsize=(12, 6))
axes[0].legend(axes[0].get_lines()[::2], data['year'][plotorder])
axes[1].semilogy()
axes[1].set_ylim(1e-4, 0.011)
axes[0].set_xlim(None, 8.)
axes[0].set_ylim(5e-5, None)
fig.savefig('figout/spectraA13to19.pdf', bbox_inches='tight')

In [ ]:
# This cell keeps some older code used for annotate a spectrum. 
# I don't need that right now, but I also don't want to reinvent the wheel later.
raise NotImplementedError


fig.set_size_inches(12, 4)

c = [l.get_c() for l in ax.get_lines()[1::2]]

def annotate(text, pos, offset, color):
    ax.annotate(text, color=color, fontsize='large',
            xy=pos, xycoords='data',
            xytext=offset, textcoords='offset points',
            bbox=dict(boxstyle="round", fc="0.9"),
            arrowprops=dict(arrowstyle="->", color=color, linewidth=2,
                            connectionstyle="angle,angleA=0,angleB=90,rad=10"))

annotate('Fe 6.7 keV', (6.7, 4e-3), (10, 30), c[3])
annotate('Fe 6.62 keV', (6.62, 1.4e-2), (-10, 30), c[2])
annotate('Ca XIX triplet', (3.88, 8e-3), (-10, 30), c[3])
annotate('Ca XX', (4.1, 3e-3), (10, 30), c[3])
annotate('Si XIII triplet', (1.85, 1.5e-2), (-10, 30), c[3])
annotate('Si XIV', (2.0, 5e-3), (10, 30), c[3])

import matplotlib.patches as mpatches
arrow = mpatches.Arrow(.6, 3e-3, 1, -2e-3, width=3e-3, color='0.6')
ax.add_patch(arrow)


In [ ]:

# FIP values from NIST
# https://physics.nist.gov/cgi-bin/ASD/ie.pl?spectra=H-DS+i&units=1&at_num_out=on&el_name_out=on&shells_out=on&level_out=on&e_out=0&unc_out=on&biblio=on
'''
@Misc{NIST_ASD,
author = {A.~Kramida and {Yu.~Ralchenko} and
J.~Reader and {and NIST ASD Team}},
HOWPUBLISHED = {{NIST Atomic Spectra Database
(ver. 5.6.1), [Online]. Available:
{\tt{https://physics.nist.gov/asd}} [2018, November 30].
National Institute of Standards and Technology,
Gaithersburg, MD.}},
year = {2018},
}
'''


# 2018 flare vs non-flare

In [ ]:
ui.load_data(100, 'data/Chandra/21176_A_grp.pi')
ui.load_data(101, 'data/Chandra/21176_A_noflare_grp.pi')
ui.load_data(102, 'data/Chandra/21176_A_flare_grp.pi')

In [ ]:

for i in range(0, 3):
    ui.ungroup(100 + i)
    ui.ignore_bad(100 + i)

ui.set_analysis("energy")
ui.ignore(None, 0.3)
ui.ignore(9., None)

In [ ]:
# Set models for 2018 flare / non-flare times
# same model, other normalization

In [ ]:

for i in range(0, 3):
    ui.set_analysis("energy")
    ui.group_counts(100+i, 5)
    ui.ignore(None, 0.3)
    ui.ignore(9., None)

    ui.plot_data(100+i, overplot=True)

ax = plt.gca()
#ax.loglog()
#ax.set_xlim(.6, 8)
#ax.set_ylim(1e-4, 0.02)
ax.set_xlim(1, 7)
ax.set_ylim(1e-4, 0.02)
fig = plt.gcf()
fig.set_size_inches(14, 5)